In [116]:
# Load main packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import getpass # For identifying user/directory structure
import networkx as nx
import re
from sklearn import linear_model
%matplotlib inline

# Load local files
import sys # For reading files in other directories
if(getpass.getuser() == 'rockc_000'):
    sys.path.insert(0, 'C:/Users/rockc_000/Documents/GitHub/GithubSandbox/Python/NCAA Scripts')
    dir = 'C:/Users/rockc_000/Documents/Personal Files/Kaggle/NCAA'
if(getpass.getuser() == 'josh'):
    sys.path.insert(0, '/home/josh/Documents/Github/GithubSandbox/Python/NCAA Scripts')
    dir = '/home/josh/Documents/Personal/Kaggle/NCAA'
from make_submission import make_submission

In [2]:
# Read in data
seasons = pd.read_csv(dir + '/Seasons.csv')
teams = pd.read_csv(dir + '/Teams.csv')
regular_compact_results = pd.read_csv(dir + '/RegularSeasonCompactResults.csv')
regular_detailed_results = pd.read_csv(dir + '/RegularSeasonDetailedResults.csv')
tourney_compact_results = pd.read_csv(dir + '/TourneyCompactResults.csv')
tourney_detailed_results = pd.read_csv(dir + '/TourneyDetailedResults.csv')
tourney_seed = pd.read_csv(dir + '/TourneySeeds.csv')
tourney_slots = pd.read_csv(dir + '/TourneySlots.csv')
sample_submission = pd.read_csv(dir + '/SampleSubmission.csv')

In [3]:
# Look at the first few rows of each dataset, and their dimensions
# sample_submission.iloc[np.arange(5)]
nrow = 4
print('seasons:\n', seasons.head(nrow), '\nDimension: ', seasons.shape)
print('teams:\n', teams.head(nrow), '\nDimension: ', teams.shape)
print('regular_compact_results:\n', regular_compact_results.head(nrow),
      '\nDimension: ', regular_compact_results.shape)
print('regular_detailed_results:\n', regular_detailed_results.head(nrow),
      '\nDimension: ', regular_detailed_results.shape)
print('tourney_compact_results:\n', tourney_compact_results.head(nrow),
      '\nDimension: ', tourney_compact_results.shape)
print('tourney_detailed_results:\n', tourney_detailed_results.head(nrow),
      '\nDimension: ', tourney_detailed_results.shape)
print('tourney_seed:\n', tourney_seed.head(nrow),
      '\nDimension: ', tourney_seed.shape)
print('tourney_slots:\n', tourney_slots.head(nrow),
      '\nDimension: ', tourney_slots.shape)
print('sample_submission:\n', sample_submission.head(nrow),
      '\nDimension: ', sample_submission.shape)

seasons:
    Season     Dayzero Regionw    Regionx    Regiony    Regionz
0    1985  10/29/1984    East       West    Midwest  Southeast
1    1986  10/28/1985    East    Midwest  Southeast       West
2    1987  10/27/1986    East  Southeast    Midwest       West
3    1988  11/02/1987    East    Midwest  Southeast       West 
Dimension:  (32, 6)
teams:
    Team_Id    Team_Name
0     1101  Abilene Chr
1     1102    Air Force
2     1103        Akron
3     1104      Alabama 
Dimension:  (364, 2)
regular_compact_results:
    Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot
0    1985      20   1228      81   1328      64    N      0
1    1985      25   1106      77   1354      70    H      0
2    1985      25   1112      63   1223      56    H      0
3    1985      25   1165      70   1432      54    H      0 
Dimension:  (139920, 8)
regular_detailed_results:
    Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot  Wfgm  Wfga ...   \
0    2003      10   1104      68   1328      6

## Pagerank algorithm on historical wins

In [6]:
# Initialize DataFrame with one (meaningless) row
pagerank_score = pd.DataFrame({'Season': [1900], 'Team': [0], 'Score': [0]})
for season in seasons['Season']:
    G = nx.DiGraph()
    edges = regular_compact_results.ix[regular_compact_results['Season'] == season,
                                       ('Wteam', 'Lteam')]
    for row in np.arange(edges.shape[0]):
        G.add_edge(edges.iloc[row][1], edges.iloc[row][0])
    # Calculate the page-rank using the networkx package
    out = nx.pagerank(G)
    # Convert the page-rank scores into a DataFrame.
    out = pd.DataFrame.from_dict(out, orient='index')
    out.rename(columns={0: 'Score'}, inplace=True)
    out['Team'] = out.index
    out['Season'] = season
    pagerank_score = pagerank_score.append(out)
pagerank_score = pagerank_score.ix[pagerank_score['Team'] != 0]

## Aggregated Statistics

In [63]:
win_count = regular_compact_results.groupby(('Season', 'Wteam'))
win_count = win_count['Daynum'].count()

# Aggregate the scores for the winning team and losing team
# as well as the number of games played.
win_pts = regular_compact_results.groupby(('Season', 'Wteam'))
win_pts = pd.merge(win_pts[('Lscore', 'Wscore')].sum(),
                    pd.DataFrame(win_pts['Season'].count()),
                    left_index=True, right_index=True)
lose_pts = regular_compact_results.groupby(('Season', 'Lteam'))
lose_pts = pd.merge(lose_pts[('Lscore', 'Wscore')].sum(),
                    pd.DataFrame(lose_pts['Season'].count()),
                    left_index=True, right_index=True)

win_pts.index.set_names(('Season', 'Team'), inplace=True)
lose_pts.index.set_names(('Season', 'Team'), inplace=True)
lose_pts.rename(columns={'Lscore': 'Points_for', 'Wscore': 'Points_againt',
                         'Season': 'Game_cnt'}, inplace=True)
win_pts.rename(columns={'Wscore': 'Points_for', 'Lscore': 'Points_againt',
                        'Season': 'Game_cnt'}, inplace=True)
total_pts = win_pts.append(lose_pts)
total_pts = total_pts.groupby(total_pts.index).sum()

In [67]:
print(total_pts.iloc[0, :])
print(win_pts.loc[1985].loc[1102])
print(lose_pts.loc[1985].loc[1102])

Game_cnt           24
Points_againt    1653
Points_for       1514
Name: (1985, 1102), dtype: int64
Points_againt    305
Points_for       355
Game_cnt           5
Name: 1102, dtype: int64
Points_for       1159
Points_againt    1348
Game_cnt           19
Name: 1102, dtype: int64


In [128]:
#stat_vars = np.array(('or', 'dr', 'ast', 'to', 'stl', 'blk', 'pf'),
#                     dtype=str)
#win_vars = np.array(['W' + x for x in stat_vars])
#lose_vars = np.array(['L' + x for x in stat_vars])

detail_win = regular_detailed_results.groupby(('Season', 'Wteam'))
detail_win = detail_win[('Wor', 'Wdr', 'Wast', 'Wto', 'Wstl', 'Wblk', 'Wpf',
                         'Lor', 'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk', 'Lpf')].mean()
def col_rename(col_name):
    col_name = re.sub('W', 'for_', col_name)
    col_name = re.sub('L', 'against_', col_name)
    return(col_name)
detail_win.rename(columns=col_rename, inplace=True)

detail_lose = regular_detailed_results.groupby(('Season', 'Lteam'))
detail_lose = detail_lose[('Lor', 'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk', 'Lpf',
                           'Wor', 'Wdr', 'Wast', 'Wto', 'Wstl', 'Wblk', 'Wpf')].mean()

#detail_data =
detail_win

for_or     for_dr    for_ast     for_to    for_stl   for_blk  \
Season Wteam                                                                    
2003   1102    3.833333  19.333333  16.916667  11.083333   7.333333  2.833333   
       1103    9.384615  21.461538  17.692308  12.538462   7.307692  1.846154   
       1104   13.529412  26.411765  14.000000  13.058824   7.235294  4.176471   
       1105   14.571429  25.857143  15.857143  18.000000  11.285714  2.000000   
       1106   12.769231  28.000000  13.000000  17.692308   9.384615  3.769231   
       1107    8.714286  25.000000  11.285714  13.000000   6.000000  2.857143   
       1108   11.714286  24.785714  16.142857  16.500000   8.785714  4.857143   
       1110    9.750000  24.000000  16.250000  12.875000   7.250000  1.687500   
       1111   12.125000  28.437500  16.687500  16.812500  10.500000  5.500000   
       1112   14.840000  28.160000  18.200000  14.800000   8.440000  4.240000   
       1113   14.000000  25.222222  16.944444  13.888889   5.722222  5.333333   
       1114    9.625000  23.000000  11.562500  16.625000   6.875000  2.750000   
       1115    9.750000  23.500000  11.000000  18.000000   8.000000  4.000000   
       1116   15.222222  28.111111  12.666667  17.666667   6.333333  4.333333   
       1117   11.636364  21.636364  14.000000  14.000000   8.454545  1.545455   
       1119    6.000000  23.333333  14.333333  17.666667   6.000000  1.333333   
       1120   12.210526  25.052632  15.421053  15.473684   8.947368  5.789474   
       1122   10.400000  25.150000  13.950000  14.750000   6.950000  4.600000   
       1123   13.076923  24.692308  13.076923  12.076923   5.076923  1.769231   
       1124   13.000000  27.214286  15.857143  12.500000   8.857143  3.428571   
       1125    9.187500  25.500000  19.062500  15.500000   8.687500  2.250000   
       1126    8.750000  23.125000  13.000000  16.625000   8.375000  1.375000   
       1127   12.000000  27.857143  13.785714  14.714286   6.142857  7.500000   
       1128   10.058824  22.647059  14.294118  14.941176   8.176471  4.294118   
       1129   10.500000  22.333333  13.000000  14.416667   7.083333  4.166667   
       1130   12.777778  22.777778  18.333333  13.111111   7.444444  2.833333   
       1131   10.350000  25.900000  14.050000  13.050000   5.900000  4.150000   
       1132    7.545455  23.181818  14.363636  15.181818   7.727273  2.909091   
       1133   10.454545  24.636364  14.545455  10.818182   9.000000  1.545455   
       1135    8.352941  24.352941  15.647059  13.941176   8.058824  3.529412   
...                 ...        ...        ...        ...        ...       ...   
2015   1433   11.076923  22.153846  14.076923  10.500000  10.423077  4.307692   
       1434    9.640000  23.720000  12.920000  11.880000   5.880000  4.560000   
       1435    8.444444  24.333333  16.500000  12.055556   4.222222  5.111111   
       1436    6.294118  24.647059  13.470588  12.764706   7.529412  5.235294   
       1437    8.875000  22.468750  16.343750  10.406250   7.937500  4.062500   
       1438    9.275862  24.586207  13.172414   9.137931   5.482759  4.551724   
       1439    7.818182  22.363636  12.454545  11.909091   4.818182  5.181818   
       1440   12.444444  23.000000  14.666667  12.666667   9.222222  5.777778   
       1441    9.000000  23.076923  11.692308  12.615385   8.384615  4.615385   
       1442    7.666667  25.333333  14.666667  11.333333   5.833333  1.000000   
       1443    9.833333  25.277778  14.388889  11.833333   5.222222  3.555556   
       1444    7.555556  22.111111  13.888889  11.000000   6.500000  2.166667   
       1447   13.666667  23.666667  11.111111  13.777778   6.111111  2.888889   
       1448   11.000000  26.923077  15.461538  13.384615   7.230769  5.000000   
       1449    9.875000  24.125000  14.375000  11.375000   5.312500  6.562500   
       1450    7.769231  24.384615  17.692308  11.076923   4.230769  4.076923   
       1451    8.090909  23.181818  11.545455  1